# Agent

## Installing packages

In [1]:
%pip install -U langchain-community langgraph langchain_openai tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


## Configuring LangSmith (optional), Tavily (search engine) and model

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

os.environ["TAVILY_API_KEY"] = getpass.getpass()

os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Full functional agent

In [3]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatOpenAI(model="gpt-4o", base_url="https://models.inference.ai.azure.com")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! How's it going in San Francisco? If there's anything you need help with or if you have any questions, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 88, 'total_tokens': 119, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-25bf03cc-7814-4845-a022-5b5ea7e7fa34-0', usage_metadata={'input_tokens': 88, 'output_tokens': 31, 'total_tokens': 119, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LDfjmmswKeEkfcBQKWVeJ3GA', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'

## Step by step agent

Tavily example invocation

In [4]:
search_results = search.invoke("what is the weather in SF")
print(search_results)

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1733665427, 'localtime': '2024-12-08 05:43'}, 'current': {'last_updated_epoch': 1733664600, 'last_updated': '2024-12-08 05:30', 'temp_c': 10.1, 'temp_f': 50.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 9, 'wind_dir': 'N', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 75, 'feelslike_c': 9.3, 'feelslike_f': 48.7, 'windchill_c': 11.5, 'windchill_f': 52.7, 'heatindex_c': 11.1, 'heatindex_f': 52.1, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 0.0, 'gust_mph': 7.9, 'gust_kph': 12.7}}"}, {'url': 'https://www.timeanddate.com/weather/@z-us-94104/

Model example invocation

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

Binding the tools to the model

In [6]:
model_with_tools = model.bind_tools(tools)

Example without the tool

In [7]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Example with the tool

In [8]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_TTOArqXg1SljJPCektbhoSa4', 'type': 'tool_call'}]


Example with predefined function: `create_react_agent` will call `.bind_tools` under the hood

In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

Example without tool

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='e3d2e8af-5075-4158-b030-3ed2673ded51'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc38eea7-ab66-43c7-a9d7-eeaf348ed92d-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91, 'input_token_details': {}, 'output_token_details': {}})]

Example with tool

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='e0ce58c8-f8d3-4b2b-9a05-54aca2cbd867'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pfBUH3Blsr2RwY9tY1qBllY9', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c78fde3a-8189-4062-81a6-591fca8586ce-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_pfBUH3Blsr2RwY9tY1qBllY9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 21, 'total_tokens': 107, 'input_token_details': {}, 'output

Streaming the response

In [12]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cMNBFV1Oar7Yy600rTONKKM8', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8b02f8a9-2d3c-4c6f-90d4-3f1825c26354-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_cMNBFV1Oar7Yy600rTONKKM8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weathera

Streming the events with `.astream_events`

In [13]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'San Francisco weather today'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1733665403, \'localtime\': \'2024-12-08 05:43\'}, \'current\': {\'last_updated_epoch\': 1733664600, \'last_updated\': \'2024-12-08 05:30\', \'temp_c\': 10.1, \'temp_f\': 50.2, \'is_day\': 0, \'condition\': {\'text\': \'Mist\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/143.png\', \'code\': 1030}, \'wind_mph\': 4.5, \'wind_kph\': 7.2, \'wind_degree\': 9, \'wind_dir\': \'N\', \'pressure_mb\': 1021.0, \'pressure_in\': 30.15, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 93, \'cloud\': 75, \'feelslike_c\': 9.3, \'feelslike_f\': 48.7, \'windchill_c\': 11.5

Definining memory

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

Adding memory to the agent  
To give memory to the agent we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [15]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

Example with memory

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-7cda88a1-1361-437d-9fe9-a97026f8bca1-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 106, 'total_tokens': 112, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e848093-f8b2-401d-bb68-156afc77054a-0', usage_metadata={'input_tokens': 106, 'output_tokens': 6, 'total_tokens': 112, 'input_token_details': {}, 'output_token_details': {}})]}}
----
